# 处理观测资源数据，并将其入库

数据情况：
- 《整合版Instruments 2022-12-04.xlsx》
- 《整合版Satellites 2022-12-04.xlsx》
- 《sensor_satellite.xlsx》


## 一、处理platform数据

### 1 创建文件，并写入前缀

In [ ]:
# 创建一个ttl文件，文件名为filename,并向文件写入prefix
def ttl_create(filename, prefix):
    relative_path = "./rdf/"  # 新创建的txt文件的存放路径
    full_path = relative_path + filename + '.ttl'  
    file = open(full_path, 'w')
    file.write(prefix)   #msg也就是下面的Hello world!
    file.close()

name="platform实例"
prefix='''@prefix : <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schema.org/> .
@base <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology> .

'''


ttl_create(name, prefix)
# 调用函数创建一个名为mytxtfile的.txt文件，并向其写入Hello world!

读取文件《整合版Satellites 2022-12-04.xlsx》

In [ ]:
import pandas as pd
path='./sensors/整合数据/整合版Satellites 2022-12-04.xlsx'
df_platform=pd.read_excel(path,sheet_name='All_Satellite')
df_platform.head()

### 2 创建“发射日期”实例

In [ ]:
# 发射日期去重
date_list=df_platform["date_of_launch"].tolist()
# print(len(date_list))
date_list=list(set(date_list))
print(len(date_list))
print(str(date_list)) 
# 该日期列为Timestamp格式

In [ ]:

## 创建日期实例
def transfer_dateIndividual(date):   # date数据类型为Timestamp
    Y=date.year
    M=date.month
    D=date.day
    dateStr=date.strftime('%Y%m%d')
    text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{dateStr}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{dateStr}> 
rdf:type owl:NamedIndividual ,time:Instant ;
time:day {D} ;
time:month {M} ;
time:year {Y} .

'''
    return text


def create_dateIndividual(filename,date_list):
    with open("./rdf/灾害事件实例.ttl", "r",encoding="utf-8") as f:  #打开文本
        origin_file = f.read()   #读取文本
        f.close()

    file = open(filename, 'a')


    # 遍历datelist
    for i in range(len(date_list)):
        dateindividual="http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#"+date_list[i].strftime('%Y%m%d');
        # 先检查有没有已存在的
        if dateindividual in origin_file:
            break
        dateIndi_text=transfer_dateIndividual(date_list[i])
        file.write(dateIndi_text)   #msg也就是下面的Hello world!
    
    file.close()

# transfer_dateIndividual(date_list[3])

filename="./rdf/platform实例.ttl"
create_dateIndividual(filename,date_list)



### 3 创建platform实例

dataProperty
- platformAgency
- platformCountry
- platformEOL
- platformFullName
- platformIdentifier
- platformInstrument
- platformObjApp
- platformRepeatCycle
- platformShortName
- rdfs:label                  

object Property
- hasOrbitType
- hasPurpose
- hasUser
- launchDate

In [ ]:
df_platform.columns
print(df_platform['expected_lifetime'][0])
if not pd.isnull(df_platform['expected_lifetime'][0]):
    print("feikong")
else:
    print("kong")

df_platform['instrument']

In [ ]:

def create_platformIndividual(df):
    text="";
    for index, row in df.iterrows():
        # 以下各个字段中如果有换行的需要去掉换行符
        identifier=row['identifier']                # 非空  string
        shortName=row['shortName']                  # 非空  string
        fullName=row['fullname_satellite']          # 非空  string
        agency=row['Agency'].replace("/",", ")                        # 非空  string  其中斜杠表示多个agency，统一替换为英文逗号。
        country=row['country_of_operator_owner'].replace("/",", ")   # 非空  string  其中斜杠表示多个country，统一替换为英文逗号。
        eol=row['expected_lifetime']                # float型，有空白   
        instrument=row['instrument']                # string 有空白  其中的分号表示多个instrument，统一替换为逗号
        objApp=row['objectives_and_applications']   # string 有空白   大段文字
        repeatCycle=row['repeat_cycle']             # int   有空白

        launchDate=row['date_of_launch'].strftime('%Y%m%d')      # date  非空
        orbitType=row['class_of_orbit'].replace("/",", :")       # enum  非空  需要把斜杠/替换为" , :"
        purpose=row['purpose'].replace("/",", :").replace(" ","")                # enum  非空  需要把斜杠/替换为" , :"
        user=row['users'].replace("/",", :")                     # enum  非空  需要把斜杠/替换为" , :"

        platform_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
:{identifier} rdf:type owl:NamedIndividual ,
                   :Platform ;
          :hasOrbitType :{orbitType} ;
          :hasPurpose :{purpose} ;
          :hasUser :{user} ;
          :launchDate <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{launchDate}> ;
          :platformAgency "{agency}" ;
          :platformCountry "{country}" ;
          :platformFullName "{fullName}" ;
          :platformIdentifier "{identifier}" ;
          :platformShortName "{shortName}" ;
          rdfs:label "{identifier}" .
'''
        if not pd.isnull(eol):
            platform_text=platform_text+f''':{identifier} :platformEOL {eol}.
            '''
        if not pd.isnull(instrument):
            instrument=instrument.replace(";",", ").replace("\n",", ")   
            platform_text=platform_text+f''':{identifier} :platformInstrument "{instrument}".
            '''
        if not pd.isnull(objApp):
            objApp=objApp.replace("\n", " ").replace("\"","'")
            platform_text=platform_text+f''':{identifier} :platformObjApp "{objApp}".
            '''
        if not pd.isnull(repeatCycle):
            platform_text=platform_text+f''':{identifier} :platformRepeatCycle {repeatCycle}.
            '''


        text=text+platform_text
    return text

text= create_platformIndividual(df_platform)
file = open("./rdf/platform实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()


## 二、处理sensor数据


读取文件

In [1]:
import pandas as pd

path='./sensors/整合数据/整合版Instruments 2022-12-04.xlsx'
df_sensor=pd.read_excel(path,sheet_name='All_Sensor')
df_sensor.head(3)

,identifier,instrument_name,full_name,satellite_name,host,type_of_sensor,purpose_of_sensor,agency,usage_from,usage_to,...,best_resolution(m),coverage_cycle,swath_width,scanning_technique,spectral_range,primary_mission_objectives,secondary_mission_objectives,opportunity_objectives,Acronym,Satellites
0,PMS-3,PMS-3,Panchromatic and Multi-spectral CCD Camera - 3,Superview 1-01 (GaoJing 1-01);Superview 1-02 (...,Superview_1-03;Superview_1-04;Jason_2;Jason_3,High resolution optical imager,High resolution land observation,CAST,2017,2028,...,0.5,"Depending on strategic pointing, down to 3 days",12.0,"Pushbroom, 12 km swath stearable along-track (...",0.45-0.52 μm\n0.52-0.59 μm\n0.63-0.69 μm\n0.77...,Biomass\nFraction of Absorbed PAR (FAPAR)\nFra...,Fire fractional cover\nGlacier cover\nGlacier ...,Aerosol volcanic ash\nAerosol volcanic ash Tot...,PMS-3,\nSuperView Neo-1 01\nSuperView Neo-1 02\nSupe...
1,SCAT,SCAT,Scatterometer,CFOSat (China-France Oceanography Satellite);H...,Haiyang_2B;Himawari_8;Himawari_9,Scatterometer,Sea-surface vector wind,NSOAS,2011,2028,...,50000.0,Global in one week,1300.0,Rotating speed: 5.7 rpm; swath 1000 km. Duty c...,Ku-Band (12.5 to 18 GHz),Wind speed (near surface)\nWind vector (near s...,NaN,Leaf Area Index (LAI)\nSea-ice type\nSnow cove...,SCAT,\nHY-2A\nHY-2B\nHY-2D\nHY-2E
2,SWIM,SWIM,Surface Waves Investigation and Monitoring ins...,CFOSat (China-France Oceanography Satellite),GOES_14,Radar altimeter,"Wave spectra (power distribution, function of ...",CNES,2019,2027,...,18000.0,Global in one week,140.0,"6 beams, incidence angles 0°, 2.4°, 4°, 6°, 8°...",\nKu-Band (12.5 to 18 GHz),Sea-ice elevation\nSea-ice thickness,Coastal sea level (tide)\nGeoid\nOcean dynamic...,Gravity field\nWind speed (near surface),SWIM,\nCFOSAT\nCFOSAT follow-on


In [ ]:
df_sensor.columns

### 1 创建年份日期实例

In [ ]:
date_from=df_sensor['usage_from'].tolist()
date_to=df_sensor['usage_to'].tolist()
date_list=list(set(date_from+date_to))
print(date_list[1])

In [ ]:
## 创建日期实例
def transfer_yearIndividual(year):   # date数据类型为int ,表示年份
    text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{year}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{year}> rdf:type owl:NamedIndividual ,
                                                                                            time:Instant ;
                                                                                   time:year {year} .

'''
    return text

def create_yearIndividual(filename,date_list):
    file = open(filename, 'a')
    # 遍历datelist
    for i in range(len(date_list)):
        dateIndi_text=transfer_yearIndividual(date_list[i])
        file.write(dateIndi_text)  
    file.close()

filename="./rdf/platform实例.ttl"
create_yearIndividual(filename,date_list)


### 2 创建sensor实例

15个数据属性，2个对象属性，有点多哦

In [ ]:
df_sensor.columns

In [ ]:
def create_sensorIndividual(df):
    text="";
    for index, row in df.iterrows():
        # 以下各个字段中如果有换行的需要去掉换行符
        identifier=row['identifier']                # 非空  string
        shortName=row['instrument_name']            # 非空  string
        fullName=row['full_name']                   # 非空  string
        sensorType=row['type_of_sensor']            # 非空  string
        
        usageFrom=row['usage_from']                 # 非空，  instant
        usageTo=row['usage_to']                     # 非空，  instant


        purpose=row['purpose_of_sensor']            # 有空白  string
        agency=row['agency']                        # 有空白  string  
        resolution=row['resolution']                # 有空白   string
        bestResolution=row['best_resolution(m)']    # 有空白   float
        revisitTime=row['coverage_cycle']           # 有空白   string
        swathWidth=row['swath_width']               # 有空白   float
        scanTech=row['scanning_technique']          # 有空白   string
        spectralRange=row['spectral_range']         # 有空白   string   有回车跨行
        primary=row['primary_mission_objectives']       # 有空白   string   有回车跨行
        secondary=row['secondary_mission_objectives']   # 有空白   string   有回车跨行
        opportunity=row['opportunity_objectives']       # 有空白   string   有回车跨行

        sensor_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
:{identifier} rdf:type owl:NamedIndividual ,
                :RSSensor ;
       :usageFrom <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{usageFrom}> ;
       :usageTo <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{usageTo}> ;
       :sensorShortName "{shortName}" ;
       :sensorFullName "{fullName}" ;
       :sensorType "{sensorType}" ;
       :sensorIdentifier "{identifier}" ;
       rdfs:label "{identifier}" .
'''

        if not pd.isnull(purpose):
            sensor_text=sensor_text+f''':{identifier} :sensorPurpose "{purpose}".
'''
        if not pd.isnull(agency):
            sensor_text=sensor_text+f''':{identifier} :sensorAgency "{agency}".
'''
        if not pd.isnull(resolution):
            sensor_text=sensor_text+f''':{identifier} :resolution """{resolution}""".
'''
        if not pd.isnull(bestResolution):
            sensor_text=sensor_text+f''':{identifier} :bestResolution {bestResolution}.
'''
        if not pd.isnull(revisitTime):
            sensor_text=sensor_text+f''':{identifier} :revisitTime "{revisitTime}".
'''
        if not pd.isnull(swathWidth):
            sensor_text=sensor_text+f''':{identifier} :swathWidth {swathWidth}.
'''
        if not pd.isnull(scanTech):
            sensor_text=sensor_text+f''':{identifier} :scanningTech """{scanTech}""".
'''
        if not pd.isnull(spectralRange):
            sensor_text=sensor_text+f''':{identifier} :spectralRange """{spectralRange}""".
'''
        if not pd.isnull(primary):
            sensor_text=sensor_text+f''':{identifier} :primaryMission """{primary}""".
'''
        if not pd.isnull(secondary):
            sensor_text=sensor_text+f''':{identifier} :secondaryMission """{secondary}""".
'''
        if not pd.isnull(opportunity):
            sensor_text=sensor_text+f''':{identifier} :opportunityMission """{opportunity}""".
'''

        text=text+sensor_text
    return text

text= create_sensorIndividual(df_sensor)
file = open("./rdf/platform实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()


## 三、添加platform和sensor之间的关系


### 1 platform  hosts sensor

In [ ]:
import pandas as pd

path='./sensors/整合数据/sensor_satellite.xlsx'
df_host=pd.read_excel(path,sheet_name='satellite')
df_host.head(3)

In [ ]:

def create_hosts_relation(df):
    text="";
    for index, row in df.iterrows():
        # 以下各个字段中如果有换行的需要去掉换行符
        platform=row['satellite_identifier']                # 非空  string
        sensors=row['hosts'].replace(";",", :")             #  非空 string


        host_text=f'''
:{platform} <http://www.w3.org/ns/sosa/hosts> :{sensors} .'''

        text=text+host_text
    return text

text= create_hosts_relation(df_host)
file = open("./rdf/platform实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()


### 2 sensor is_hosted_by platform
http://www.w3.org/ns/sosa/isHostedBy

In [ ]:
import pandas as pd

path='./sensors/整合数据/sensor_satellite.xlsx'
df_isHostedBy=pd.read_excel(path,sheet_name='sensor')
df_isHostedBy.head(3)

In [ ]:
def create_isHostedBy_relation(df):
    text="";
    for index, row in df.iterrows():
        # 以下各个字段中如果有换行的需要去掉换行符
        sensor=row['instrument_identifier']                # 非空  string
        platform=row['is_host_by'].replace(";",", :")      #  非空 string


        isHostedBy_text=f'''
:{sensor} <http://www.w3.org/ns/sosa/hosts> :{platform} .'''

        text=text+isHostedBy_text
    return text

text= create_isHostedBy_relation(df_isHostedBy)
file = open("./rdf/platform实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()


## 四、处理气象站数据


读取数据《气象站整合数据.xlsx》

In [65]:
import pandas as pd

path='./sensors/气象站数据/气象站整合数据.xlsx'
df_station=pd.read_excel(path,sheet_name='integrated',dtype={"开始年份": "string","usageTo":"string","缺测时段(duration)":"string"},keep_default_na=False)
df_station.head(3)

,省份,区站号,站名,经度\n（度分）,lon,纬度\n（度分）,lat,pointID,WKT,观测场海拔高度（米）,气压传感器海拔高度（米）,站类,开始年份,usageTo,结束年份,缺测时段(duration),（临时）标准值统计时段(duration),obsFrequency\n(interval),obsFeature
0,黑龙江,50136,漠河,12231,122.516667,5258,52.966667,122.5167_52.9667,POINT Z (122.5167 52.9667 438.5),438.5,439.7,基准站,198101,,201212,,198101_199612/199701_201012,,
1,黑龙江,50137,北极村,12222,122.366667,5328,53.466667,122.3667_53.4667,POINT Z (122.3667 53.4667 296.0),296.0,297.3,一般站,200201,,201212,,,,
2,黑龙江,50246,塔河,12443,124.716667,5221,52.350000,124.7167_52.3500,POINT Z (124.7167 52.3500 361.9),361.9,363.0,基本站,198101,,201212,,198101_201012,,


创建文件，并写入前缀

In [ ]:
# 创建一个ttl文件，文件名为filename,并向文件写入prefix
def ttl_create(filename, prefix):
    relative_path = "./rdf/"  # 新创建的txt文件的存放路径
    full_path = relative_path + filename + '.ttl'  
    file = open(full_path, 'w')
    file.write(prefix)   #msg也就是下面的Hello world!
    file.close()

name="气象站点观测资源实例"
prefix='''@prefix : <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schema.org/> .
@base <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology> .

'''


ttl_create(name, prefix)
# 调用函数创建一个名为mytxtfile的.txt文件，并向其写入Hello world!

### 1 创建日期实例（usageFrom）

In [ ]:
ym_list=df_station['开始年份'].tolist()+df_station['usageTo'].tolist()
ym_list=list(set(ym_list))
ym_list.remove('')
ym_list


In [ ]:
# 创建日期实例
# dateStr为yyyymm文本格式
def transfer_dateIndividual(dateStr):
    Y=dateStr[0:4]
    M=int(dateStr[4:6])
    
    text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{dateStr}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{dateStr}> 
rdf:type owl:NamedIndividual ,time:Instant ;
time:month {M} ;
time:year {Y} .

'''
    return text


def create_dateIndividual(filename,date_list):
    with open("./rdf/原文件.ttl", "r",encoding="utf-8") as f:  #打开文本
        origin_file = f.read()   #读取文本
        f.close()

    file = open(filename, 'a')


    # 遍历datelist
    for i in range(len(date_list)):
        dateindividual="http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#"+date_list[i];
        # 先检查有没有已存在的
        if dateindividual in origin_file:
            break
        dateIndi_text=transfer_dateIndividual(date_list[i])
        file.write(dateIndi_text)   
    
    file.close()

create_dateIndividual("./rdf/气象站点观测资源实例.ttl",ym_list)

### 2 创建时间段实例（缺测时段）

In [ ]:
missperiod_list=df_station['缺测时段(duration)'].tolist()

for item in iter(missperiod_list):
    if ',' in item:
        temp_list=item.split(",")
        missperiod_list=missperiod_list+temp_list;
        missperiod_list.remove(item)
missperiod_list=list(set(missperiod_list))
missperiod_list.remove('')
list(set(missperiod_list))

In [63]:
def create_IntervalIndividual(filename,interval_list):
    file=open(filename,'a')

    for i in range(len(interval_list)):
        interval=interval_list[i]
        identifier=interval+"UTC+8"
        if "_" in interval:
            startY=interval[0:4]
            startM=interval[4:6]
            endY=interval[7:11]
            endM=interval[11:13]
            text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}> rdf:type owl:NamedIndividual, time:Interval ;
rdfs:label "{startY}-{startM} to {endY}-{endM} UTC+8:00" .

'''
        else:
            Y=interval[0:4]
            M=interval[4:6]
            text=f'''###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}> rdf:type owl:NamedIndividual, time:Interval ;
rdfs:label "{Y}-{M} UTC+8:00" .

'''
        # print(text)
        file.write(text)
    file.close();


create_IntervalIndividual("./rdf/气象站点观测资源实例.ttl",missperiod_list)

### 3 创建空间实例（Point）
pointID字段为ID

WKT字段为其空间几何

In [64]:
df_station.columns

Index(['省份', '区站号', '站名', '经度\n（度分）', 'lon', '纬度\n（度分）', 'lat', 'pointID',
       'WKT', '观测场海拔高度（米）', '气压传感器海拔高度（米）', '站类', '开始年份', 'usageTo', '结束年份',
       '缺测时段(duration)', '（临时）标准值统计时段(duration)', 'obsFrequency\n(interval)',
       'obsFeature'],
      dtype='object')

### 4 创建气象站实例

In [71]:
def create_stationIndividual(df):
    text="";
    for index, row in df.iterrows():
        # 以下各个字段中如果有换行的需要去掉换行符
        stationID=row['区站号']                 # 非空  string
        province=row['省份']                    # 非空  string
        name=row['站名']                        # 非空  string
        lon=row['lon']                         # 非空  float 
        lat=row['lat']                         # 非空  float
        alt=row['观测场海拔高度（米）']          # 非空  string
        pointID=row["pointID"]                 # 非空 string
        wkt=row['WKT']                         # 非空 string
        stationType=row['站类']                # 有空 string，需要相应地转换
        usageFrom=row['开始年份']              # 有空 string
        usageTo = row['usageTo']               # 有空 string
        interval=row['缺测时段(duration)']      # 有空 string
        
        station_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#point.{pointID}
:point.{pointID} rdf:type owl:NamedIndividual ,
                                 <http://www.opengis.net/ont/sf#Point> ;
                        <http://www.opengis.net/ont/geosparql#asWKT> "{wkt}"^^<http://www.opengis.net/ont/geosparql#wktLiteral> ;
                        rdfs:label "Point {lon} {lat} {alt}" .


###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{stationID}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{stationID}> rdf:type owl:NamedIndividual ,
                                                                                             <http://www.opengis.net/ont/geosparql#Feature> ,
                                                                                             :In-situSensor ;
                                                                                    <http://www.opengis.net/ont/geosparql#hasGeometry> :point.{pointID} ;
                                                                                    :locationAlt {alt} ;
                                                                                    :locationLat {lat} ;
                                                                                    :locationLon {lon} ;
                                                                                    :sensorAgency "{province}" ;
                                                                                    :sensorFullName "{name}" ;
                                                                                    :sensorIdentifier {stationID} ;
                                                                                    :sensorShortName "{name}" ;
                                                                                    :sensorType "{stationType}" ;
                                                                                    rdfs:label "{name}" .
'''
        if not stationType=="":
            stype=""
            if stationType == "一般站":
                stype="GeneralStation"
            elif stationType == "基本站":
                stype="BaseStation"
            elif stationType == "基准站":
                stype="ReferenceStation"
            station_text=station_text+f'''<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{stationID}> :hasStationType :{stype};
            :sensorType "{stationType}" .
'''
        if not usageFrom=="": 
            station_text=station_text+f'''<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{stationID}> :usageFrom <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{usageFrom}> .
'''
        if not usageTo=="":
            station_text=station_text+f'''<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{stationID}> :usageTo <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{usageTo}> .
'''
        if not interval== "":
            interval_id=interval+"UTC+8"
            station_text=station_text+f'''<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{stationID}> :missingPeriod <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{interval_id}> .
'''


        text=text+station_text
    return text

text= create_stationIndividual(df_station)
file = open("./rdf/气象站点观测资源实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()